# Unit 6. Essential Deep Learning Tips & Tricks

## 6.1 Model Checkpointing and Early Stopping

## Part 2. Preamble: Basic & General Dataset Inspection

## 1) Installing Libraries

In [1]:
# !conda install jupyterlab numpy pandas matplotlib watermark sklearn --yes

In [2]:
# !pip install torch torchvision torchaudio

In [3]:
# !pip install lightning

In [4]:
%load_ext watermark
%watermark -v -p numpy,pandas,matplotlib,torch,lightning,scikit-learn --conda

Python implementation: CPython
Python version       : 3.9.16
IPython version      : 8.11.0

numpy       : 1.24.2
pandas      : 1.5.3
matplotlib  : 3.7.1
torch       : 1.13.1
lightning   : 1.9.4
scikit-learn: 1.2.1

conda environment: dl-fundamentals



## 2) Loading the dataset

In [5]:
from shared_utilities import CustomDataModule

dm = CustomDataModule()
dm.setup("train")

In [6]:
print("Training set size:", len(dm.train_dataset))
print("Validation set size:", len(dm.val_dataset))
print("Test set size:", len(dm.test_dataset))

Training set size: 14400
Validation set size: 1600
Test set size: 4000


### Check label distribution

In [7]:
from collections import Counter

train_loader = dm.train_dataloader()
val_loader = dm.val_dataloader()
test_loader = dm.test_dataloader()

train_counter = Counter()
for features, labels in train_loader:
    train_counter.update(labels.tolist())
    
print("\nTraining label distribution:")
print(sorted(train_counter.items()))

    
val_counter = Counter()
for features, labels in val_loader:
    val_counter.update(labels.tolist())
    
print("\nValidation label distribution:")
print(sorted(val_counter.items()))
    

test_counter = Counter()
for features, labels in test_loader:
    test_counter.update(labels.tolist())

print("\nTest label distribution:")
print(sorted(test_counter.items()))


Training label distribution:
[(0, 7209), (1, 7191)]

Validation label distribution:
[(0, 791), (1, 809)]

Test label distribution:
[(0, 2027), (1, 1973)]


## 3) Zero-rule baseline (majority class classifier)

In [8]:
majority_class = test_counter.most_common(1)[0]
print("Majority class:", majority_class[0])

baseline_acc = majority_class[1] / sum(test_counter.values())
print("Accuracy when always predicting the majority class:")
print(f"{baseline_acc:.2f} ({baseline_acc*100:.2f}%)")

Majority class: 0
Accuracy when always predicting the majority class:
0.51 (50.68%)


## 4) A quick visual check

In [9]:
features.shape

torch.Size([32, 100])

In [10]:
features[0] # first training example

tensor([-1.0024, -0.1871, -0.0753,  1.0124,  1.2639, -1.9450, -4.4824, -3.4725,
         3.7921,  0.2157, -1.6438,  4.2781, -2.3388, -0.8952, -1.4974, -4.3888,
         0.2676,  0.5185, -1.9450,  1.1234,  0.0289,  1.5448, -4.2949,  0.8634,
        -2.5792,  4.8754,  0.8249, -4.4946,  0.9226, -4.4946,  0.0597, -2.9464,
        -2.1122, -2.4712, -0.1437,  1.6721, -0.7004, -1.0092, -3.2290,  2.3953,
        -2.3388, -4.4824,  4.1454,  2.6818, -1.4974,  0.5100,  2.7505, -0.9028,
        -1.3736, -0.7225,  0.5747, -4.4824, -2.0825,  0.8780,  3.2301,  1.0947,
         0.8193, -4.6827, -0.0749,  0.5185, -2.5792, -0.8514, -0.0891, -1.3285,
        -2.1097, -4.4824, -4.6827, -0.1336,  0.1203,  2.3953,  4.2781, -1.0752,
        -2.2972, -4.4946,  0.9662, -5.0314, -1.8789, -0.9486, -1.0092,  1.1786,
         0.7567, -2.0825,  0.7605,  3.2301, -2.5792, -3.2290,  0.5185, -4.2949,
        -2.5792, -1.8789,  0.5185, -3.4725, -1.0276, -0.1871, -0.5376,  1.0599,
         0.8794,  0.0504, -1.8460, -1.25